In [ ]:
!pip install lxml

In [ ]:
!pip install bs4

In [7]:
from scrapy.spiders import SitemapSpider

In [8]:
from scrapy.spiders import CrawlSpider
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

In [9]:
!pwd

/home/alex/chat_bot_wrike/wrike-bot


In [10]:
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
import math 
from nltk.corpus import wordnet
import urllib.request
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import re
import requests as req
import pandas as pd
import numpy as np

import pymorphy2
nltk.download('stopwords')
english_stopwords = stopwords.words("english")
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

def clean_data(x): 
    g_text1 = '©2006 Wrike, Inc. All rights reserved. Patented. Privacy Policy . Terms of Service . CA Notice at Collection .  English (US)  Deutsch  Español  Français  Italiano  日本語  Português do Brasil  Русский      Follow us                           ©2006 Wrike, Inc. All rights reserved. Patented. Privacy Policy & Terms of Service    Community Welcome 🖖  Hi there! 🙂 Want to become a black belt Wrike Ninja? Here\'s how to earn a Wrike badge  Welcome 🖖 Have you checked out this week\'s Release Notes yet?  Hey! 👋 Curious about something? Visit How To to search and ask the Community for answers.  Welcome! 👋 Figured out a good tip or trick? Share it in Best Practices .  Want to connect your existing software to Wrike? Learn and ask how in the API section .'
    g_text2 = 'Oops we\'re sorry! It looks like we don\'t have the translated page you\'re looking for, but we do have other content in English and offer support in English. Browse Help Center in English Submit a Request    Hey there! 👋 It looks like you\'re trying to figure something out. Stuck or just\n      want advice? Search the Community   Greetings 🖖 You can unlock the true value of Wrike in our Community Connect with thousands of leading teams, learn best practices, and shape\n      the future of the product. What are you waiting for? Sign-in\n        with Wrike Start a free\n        Wrike Trial?           Submit a request  Sign in    What can we help you with? e.g. Gantt chart , Creating tasks , Sharing folders    Search   Community Only   Call us: Show number  Blog English (US)  Deutsch  Español  Français  Italiano  日本語  Português do Brasil  Русский'
    g_text3 = 'Click the "Follow" button in the task'
    g_text4 = '\xa0'
    x = x.replace(g_text1, " ")
    x = x.replace(g_text2, " ")
    x = x.replace(g_text3, " ")
    x = x.replace(g_text4, " ")
    x = re.sub('[\n\r]', '', x)
    x = re.sub(' +', ' ',x)
    return x 

[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/alex/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [11]:
import copy 

def preprocess_text(text_copy, tokenizer=RegexpTokenizer(r'\w+'), 
                    lemmatizer=WordNetLemmatizer(), stopwords=stopwords.words("english"), pure = False):
    text = copy.deepcopy(text_copy)
    tokens = tokenizer.tokenize(text)
    
    if pure: 
        tokens = [token.lower() for token in tokens]
        tokens = [token for token in tokens if token not in english_stopwords\
                  and token != " " \
                  and not token.isdigit()]
        lemmas = [lemmatizer.lemmatize(token) for token in tokens]
        clean_text = " ".join(lemmas)
    else: 
        clean_text = " ".join(tokens)
    return clean_text

In [12]:
link_ref = ['/hc/en-us/requests/new'
            ,'/hc/en-us/search?utf8=✓&query=Gantt+chart'
            ,'/hc/en-us/search?utf8=✓&query=Creating+tasks'
            ,'/hc/en-us/search?utf8=✓&query=Sharing+folders']


data_ref = "profilling.csv"

def tag_visible(element):
    
    tag_ = element.parent.name
    tag_p = None
    try: 
        tag_p = element.parent.parent.name
    except: 
        pass
#     try: 
#         print(element.parent.text)
#     except: 
#         return False
    if tag_ in ['style', 'script', 'head', 'title', 'meta', '[document]', 'div', 'section', 'footer']:
        return False
    
    if tag_p in ['section', 'div']:
        return False
    
    if tag_p in ["p", "nav", "span"] and tag_ == "a":
        if element.parent.get_attribute_list("href")[-1] is not None  and \
               (not element.parent.get_attribute_list("href")[-1].startswith("/hc/en-us/signin")) and \
               (not element.parent.get_attribute_list("href")[-1].startswith("/hc/en-us/community")) and \
               (element.parent.get_attribute_list("href")[-1] not in link_ref) and \
               (element.parent.get_attribute_list("href")[-1].startswith("/hc/en-us/") or\
                element.parent.get_attribute_list("href")[-1][0] == "#"): 
            r1 = element.parent.get_attribute_list("href")[-1]
            if r1.startswith("/hc/en-us/profiles"):
                with open(data_ref, "a+") as f: 
                    f.write(re.sub('/hc/en-us/profiles/(\d*-)', '\n', r1))
            return True
            
        return False
    
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'lxml')
    texts = soup.findAll(text=True)
    for a in soup.find_all('a', href=True):
    print "Found the URL:", a['href']
        return ''
    return  x

In [14]:
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy.crawler import CrawlerProcess

class TheFriendlyNeighbourhoodSpider(CrawlSpider):
    name = 'TheFriendlyNeighbourhoodSpider'
    allowed_domains = ['help.wrike.com']
    start_urls = ['https://help.wrike.com/hc/en-us/']
    custom_settings = {
    'CONCURRENT_REQUESTS': 10,
    'CONCURRENT_REQUESTS_PER_DOMAIN': 25,
    'ROBOTSTXT_OBEY': False,
    'CONCURRENT_ITEMS': 100,
    'REACTOR_THREADPOOL_MAXSIZE': 400,
    # Hides printing item dicts
    'LOG_LEVEL': 'INFO',
    'RETRY_ENABLED': False,
    'REDIRECT_MAX_TIMES': 1,
    # Grabs xpath before site finish loading
    'DOWNLOAD_FAIL_ON_DATALOSS': False,
    'DEPTH_PRIORITY': 1,
    'SCHEDULER_DISK_QUEUE' : 'scrapy.squeues.PickleFifoDiskQueue',
    'SCHEDULER_MEMORY_QUEUE' :'scrapy.squeues.FifoMemoryQueue'
    }
    rules = (
        Rule(LinkExtractor(), callback='parse_item', follow=True),
    )
    def parse_item(self, response):
        #print('Downloaded... ', response.url)
        if (not str(response.url).startswith("https://help.wrike.com/hc/en-us/signin")) and\
        ((str(response.url).startswith("https://help.wrike.com/hc/en-us")) \
                    or (str(response.url).startswith("https://www.wrike.com/blog/"))):
            data = response.url.split("/")[-1]
            if data == "": 
                data = response.url.split("/")[-2]
            filename = '/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/' + data + '.csv'
            filename_pure = '/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike_pure/' + data + '.csv'
            print('Saving as :', filename)
            html = urllib.request.urlopen(response.url).read()
            f_text_raw = text_from_html(html)
            if len(f_text_raw) > 30: 
                with open(filename_pure, 'wb') as f_pure, open(filename, 'wb') as f:
                    html = urllib.request.urlopen(response.url).read()
                    f_text_raw = text_from_html(html) 
                    f_text_pure = preprocess_text(f_text_raw, pure = True)
                    f_text = preprocess_text(f_text_raw)
                    print(filename)
                    f.write(str(str(response.url)+" \n").encode('utf_8'))
                    f.write(f_text.encode('utf_8'))
                    f_pure.write(str(str(response.url)+" \n").encode('utf_8'))
                    f_pure.write(f_text_pure.encode('utf_8'))


In [15]:
%%time
if __name__ == "__main__":
    process = CrawlerProcess()
    process.crawl(TheFriendlyNeighbourhoodSpider)
    process.start()
    process.stop()

2020-09-12 16:01:01 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-09-12 16:01:01 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.1 (default, Dec 14 2018, 19:28:38) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.4.2, Platform Linux-5.4.0-45-generic-x86_64-with-debian-buster-sid
2020-09-12 16:01:01 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-09-12 16:01:01 [scrapy.crawler] INFO: Overridden settings:
{'CONCURRENT_REQUESTS': 10,
 'CONCURRENT_REQUESTS_PER_DOMAIN': 25,
 'DEPTH_PRIORITY': 1,
 'DOWNLOAD_FAIL_ON_DATALOSS': False,
 'LOG_LEVEL': 'INFO',
 'REACTOR_THREADPOOL_MAXSIZE': 400,
 'REDIRECT_MAX_TIMES': 1,
 'RETRY_ENABLED': False,
 'SCHEDULER_DISK_QUEUE': 'scrapy.squeues.PickleFifoDiskQueue',
 'SCHEDULER_MEMORY_QUEUE': 'scrapy.squeues.FifoMemoryQueue'}
2020-09-12 16:01:01 [scrapy.extensions.telnet] INFO: T

Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/en-us.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/en-us.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/new.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?utf8=%E2%9C%93&query=Creating+tasks.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?utf8=%E2%9C%93&query=Creating+tasks.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?utf8=%E2%9C%93&query=Gantt+chart.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?utf8=%E2%9C%93&query=Gantt+chart.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?utf8=%E2%9C%93&query=Sharing+folders.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?utf8=%E2%9C%93&query=Sharing+folders.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/topics.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/topics.csv
Saving as : /home

2020-09-12 16:02:09 [scrapy.extensions.logstats] INFO: Crawled 95 pages (at 95 pages/min), scraped 0 items (at 0 items/min)


/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?page=6&query=Creating+tasks&utf8=%E2%9C%93.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/360001383634--Duplicate-Create-Task-with-Subtask-Templates-to-generate-from-Request-Forms.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/360001383634--Duplicate-Create-Task-with-Subtask-Templates-to-generate-from-Request-Forms.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?page=100&query=Creating+tasks&utf8=%E2%9C%93.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?page=100&query=Creating+tasks&utf8=%E2%9C%93.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?filter_by=knowledge_base&query=Gantt+chart&utf8=%E2%9C%93.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?filter_by=knowledge_base&query=Gantt+chart&utf8=%E2%9C%93.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/search?filter_by=community&query=Gan

2020-09-12 16:03:08 [scrapy.extensions.logstats] INFO: Crawled 155 pages (at 60 pages/min), scraped 0 items (at 0 items/min)


...

Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/383229098674-Kristin-Gordon.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/383229098674-Kristin-Gordon.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/360026344093-Calendar-dropdown-organization.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/360026344093-Calendar-dropdown-organization.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/360026121453-Request-form-assign-Blueprint-based-on-question-selection.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/360026121453-Request-form-assign-Blueprint-based-on-question-selection.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/360018501914-Single-task-Gannt-view.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/360018501914-Single-task-Gannt-view.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/1507947652-Lee-Elkins?filter_by=posts&sort_by=votes.csv
/home/alex/chat_b

2020-09-12 23:02:56 [scrapy.crawler] INFO: Received SIGINT, shutting down gracefully. Send again to force 
2020-09-12 23:02:57 [scrapy.core.engine] INFO: Closing spider (shutdown)


/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/14766648265-Ryan-Monson?filter_by=posts&sort_by=recent_user_activity.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/14345412025-Jeremy-Foster?filter_by=posts&sort_by=recent_user_activity.csv
/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/14345412025-Jeremy-Foster?filter_by=posts&sort_by=recent_user_activity.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/14345412025-Jeremy-Foster?filter_by=comments&sort_by=recent_user_activity.csv


2020-09-12 23:02:58 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/exception_count': 4,
 'downloader/exception_type_count/twisted.web._newclient.ResponseNeverReceived': 4,
 'downloader/request_bytes': 1067896382,
 'downloader/request_count': 233670,
 'downloader/request_method_count/GET': 233670,
 'downloader/response_bytes': 2662483583,
 'downloader/response_count': 233666,
 'downloader/response_status_count/200': 100325,
 'downloader/response_status_count/301': 57917,
 'downloader/response_status_count/302': 44239,
 'downloader/response_status_count/404': 11281,
 'downloader/response_status_count/429': 19902,
 'downloader/response_status_count/520': 1,
 'downloader/response_status_count/524': 1,
 'dupefilter/filtered': 2839210,
 'elapsed_time_seconds': 25316.791277,
 'finish_reason': 'shutdown',
 'finish_time': datetime.datetime(2020, 9, 12, 20, 2, 58, 599104),
 'log_count/ERROR': 33,
 'log_count/INFO': 432,
 'memusage/max': 2825355264,
 'memusage/startup': 2069708

/home/alex/chat_bot_wrike/wrike-bot/help_p_for_wrike/14345412025-Jeremy-Foster?filter_by=comments&sort_by=recent_user_activity.csv
CPU times: user 1h 43min 6s, sys: 1min, total: 1h 44min 7s
Wall time: 7h 1min 58s


In [ ]:
process.stop()

Лишний текст

In [2]:
g_text1 = '©2006 Wrike, Inc. All rights reserved. Patented. Privacy Policy . Terms of Service . CA Notice at Collection .  English (US)  Deutsch  Español  Français  Italiano  日本語  Português do Brasil  Русский      Follow us                           ©2006 Wrike, Inc. All rights reserved. Patented. Privacy Policy & Terms of Service    Community Welcome 🖖  Hi there! 🙂 Want to become a black belt Wrike Ninja? Here\'s how to earn a Wrike badge  Welcome 🖖 Have you checked out this week\'s Release Notes yet?  Hey! 👋 Curious about something? Visit How To to search and ask the Community for answers.  Welcome! 👋 Figured out a good tip or trick? Share it in Best Practices .  Want to connect your existing software to Wrike? Learn and ask how in the API section .'
g_text2 = 'Oops we\'re sorry! It looks like we don\'t have the translated page you\'re looking for, but we do have other content in English and offer support in English. Browse Help Center in English Submit a Request    Hey there! 👋 It looks like you\'re trying to figure something out. Stuck or just\n      want advice? Search the Community   Greetings 🖖 You can unlock the true value of Wrike in our Community Connect with thousands of leading teams, learn best practices, and shape\n      the future of the product. What are you waiting for? Sign-in\n        with Wrike Start a free\n        Wrike Trial?           Submit a request  Sign in    What can we help you with? e.g. Gantt chart , Creating tasks , Sharing folders    Search   Community Only   Call us: Show number  Blog English (US)  Deutsch  Español  Français  Italiano  日本語  Português do Brasil  Русский'
g_text3 = 'Click the "Follow" button in the task'
g_text4 = '\xa0'

In [3]:
re.sub(r"\d+-", "", "201188645-Getting-Started")

'Getting-Started'

Возьмем все сайты, отоброжающие подразделы 

In [66]:
df_mine = pd.DataFrame(columns = ["First_stage",  "Second_stage",  "url"])
base_name = "https://help.wrike.com"
resp = req.get("https://help.wrike.com/hc/en-us")
 
soup = BeautifulSoup(resp.text, 'lxml')

first_group = ['Tasks, Folders, Projects and Spaces',
       'Types of Accounts and Licenses', 'Work Views', 'Monitoring Panel',
       'Integrations', 'Account Management', 'Security', 'Apps',
       'Getting Started']

# print(soup.title)
# print(soup.title.text)
# print(soup.title.parent)
dict_data = {}
dict_data_list = {}
dict_data_item = {}
for tag in soup.find_all("h4"):
    #print(tag.text, tag.text in first_group)
    
    if tag.text in first_group:
        data = tag.parent
        print("{:-^100}".format(tag.text))
        #print(data)
        #print(dir(data))
        dict_data[tag.text] = data.get_attribute_list("href")
        dict_data_list[tag.text] = base_name + "/" + data.get_attribute_list("href")[0]
        #print(dict_data_list[tag.text])
        resp_c = req.get(dict_data_list[tag.text])
         
        soup_c = BeautifulSoup(resp_c.text, 'lxml') 
        child_data = soup_c.find_all("a", {"class":"see-all-articles"})
        child_data_all = soup_c.find_all("a", {"class":"article-list-link"})
        child_data_all_n = soup_c.find_all("a", {"class":"getting-started__article-link"})
        for i in child_data_all:
            dict_data_item[i.text] = base_name + "/" + i.get_attribute_list("href")[0]
            df_mine.loc[len(df_mine)+1] = [tag.text, i.text, base_name + "/" + i.get_attribute_list("href")[0]]
        
        if child_data_all_n is not None: 
            for j in child_data_all_n:
                resp_co = req.get(base_name + "/" + j.get_attribute_list("href")[0])
                soup_co = BeautifulSoup(resp_co.text, 'lxml') 
                child_data_all_co = soup_co.find_all("a", {"class":"sidenav-item "})

                for i in child_data_all_co:
                    dict_data_item[i.text] = base_name + "/" + i.get_attribute_list("href")[0]
                    df_mine.loc[len(df_mine)+1] = [tag.text, i.text, base_name + "/" + i.get_attribute_list("href")[0]]

        if child_data is not None: 
            for j in child_data:
                resp_co = req.get(base_name + "/" + j.get_attribute_list("href")[0])
                soup_co = BeautifulSoup(resp_co.text, 'lxml') 
                child_data_all_co = soup_co.find_all("a", {"class":"article-list-link"})
                for i in child_data_all_co:
                    dict_data_item[i.text] = base_name + "/" + i.get_attribute_list("href")[0]
                    df_mine.loc[len(df_mine)+1] = [tag.text, i.text, base_name + "/" + i.get_attribute_list("href")[0]]

        #print(data.get_attribute_list("href"))


------------------------------------------Getting Started-------------------------------------------
--------------------------------Tasks, Folders, Projects and Spaces---------------------------------
-----------------------------------Types of Accounts and Licenses-----------------------------------
---------------------------------------------Work Views---------------------------------------------
------------------------------------------Monitoring Panel------------------------------------------
--------------------------------------------Integrations--------------------------------------------
-----------------------------------------Account Management-----------------------------------------
----------------------------------------------Security----------------------------------------------
------------------------------------------------Apps------------------------------------------------


In [67]:
df_mine.shape

(374, 3)

In [68]:
df_mine = df_mine.drop_duplicates(subset=["url"]).reset_index(drop = True)

In [69]:
df_mine.shape

(240, 3)

In [70]:
df_mine.loc[1:2]["url"].values

array(['https://help.wrike.com//hc/en-us/articles/210325145-Step-3-Team-Onboarding-and-Collaboration',
       'https://help.wrike.com//hc/en-us/articles/360040979053-Step-4-Monitoring-Work-Progress'],
      dtype=object)

In [71]:
df_mine.head(10)

,First_stage,Second_stage,url
0,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...
1,Getting Started,Step 3: Team Onboarding and Collaboration,https://help.wrike.com//hc/en-us/articles/2103...
2,Getting Started,Step 4: Monitoring Work Progress,https://help.wrike.com//hc/en-us/articles/3600...
3,Getting Started,Step 5: Tailoring Wrike to Your Team’s Needs,https://help.wrike.com//hc/en-us/articles/3600...
4,Getting Started,Step 6. Making Remote Work Productive,https://help.wrike.com//hc/en-us/articles/3600...
5,Getting Started,Step 1: Learning the Fundamentals,https://help.wrike.com//hc/en-us/articles/1150...
6,Getting Started,Step 2: Launching New Projects,https://help.wrike.com//hc/en-us/articles/2139...
7,Getting Started,Step 3: Collaborating in Wrike,https://help.wrike.com//hc/en-us/articles/2144...
8,Getting Started,Step 4: Work from Home Productively,https://help.wrike.com//hc/en-us/articles/3600...
9,Getting Started,Step 1: Getting Settled,https://help.wrike.com//hc/en-us/articles/2139...


In [72]:
l1 = '<a href="#settingup" target="_self">Setting Up Your Profile</a>' 
m = re.search(r'#\w+', l1)
if m:
    found = m.group(0)
    print(found)

#settingup


In [73]:
from tqdm import tqdm
list_data = []
for k in tqdm(range(df_mine.shape[0])):
    resp_1 = req.get(df_mine["url"].loc[k])
    soup_1 = BeautifulSoup(resp_1.text, 'html.parser') 
    #print(soup_1)  
    #print(soup_1)
    child_data_all_1 = soup_1.find_all("a", {"href":re.compile(r'^#\w+')})
    l1 = []
    for i in child_data_all_1:
        m = re.search(r'#\w+', str(i))
        if m:
            found = m.group(0)
            l1.append((i.text, found))
    list_data.append(l1)

100%|██████████| 240/240 [03:06<00:00,  1.29it/s]


In [74]:
df_mine["Articles"] = list_data

In [75]:
df_mine.head()

,First_stage,Second_stage,url,Articles
0,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...,"[(Setting Up Your Profile, #settingup), (Basic..."
1,Getting Started,Step 3: Team Onboarding and Collaboration,https://help.wrike.com//hc/en-us/articles/2103...,"[(Team Onboarding, #onboarding), (Change Manag..."
2,Getting Started,Step 4: Monitoring Work Progress,https://help.wrike.com//hc/en-us/articles/3600...,"[(Manager Dashboard, #manager), (Team Dashboar..."
3,Getting Started,Step 5: Tailoring Wrike to Your Team’s Needs,https://help.wrike.com//hc/en-us/articles/3600...,"[(Custom Workflows, #workflows), (Blueprints, ..."
4,Getting Started,Step 6. Making Remote Work Productive,https://help.wrike.com//hc/en-us/articles/3600...,"[(Overview, #overview), (Team Efficiency, #tea..."


In [76]:
df_mine = df_mine.explode('Articles').reset_index(drop=True)

In [77]:
df_mine['ArticlesName'] = df_mine['Articles'].apply(lambda x:x[0])
#df_mine = df_mine[df_mine["ArticlesName"] != "Return to top"].reset_index(drop = True)
df_mine['ArticlesUrl'] = df_mine['Articles'].apply(lambda x:x[-1].split("#")[-1])
del df_mine['Articles'] 

In [78]:
df_mine["Second_stage_next"] = df_mine["Second_stage"].shift(-1, fill_value = "")
df_mine["ArticlesUrl_next"] = df_mine["ArticlesUrl"].shift(-1, fill_value = "")
df_mine["ArticlesUrl_next"] = df_mine['ArticlesUrl_next'].where(df_mine['Second_stage'] == df_mine['Second_stage_next'], '')
del df_mine["Second_stage_next"]

In [79]:
df_mine.shape

(1681, 6)

In [80]:
df_mine.loc[0].values

array(['Getting Started', 'Step 2: Setting Up Your Workspace',
       'https://help.wrike.com//hc/en-us/articles/210325165-Step-2-Setting-Up-Your-Workspace',
       'Setting Up Your Profile', 'settingup', 'administration'],
      dtype=object)

In [51]:
# s1 = 'https://help.wrike.com//hc/en-us/articles/210325165-Step-2-Setting-Up-Your-Workspace'
# resp = req.get(s1)
# s_pattern = BeautifulSoup(resp.text) 
# s = s_pattern.findAll(text=True)

In [59]:
# s_pattern.find(id="settingup").parent

<h2><a id="settingup"></a>Setting Up Your Profile</h2>

In [85]:
df_mine = df_mine[(df_mine["ArticlesName"] != "More Resources") & (df_mine["ArticlesName"] != "Return to top")]

In [86]:
df_mine.head(6)

,First_stage,Second_stage,url,ArticlesName,ArticlesUrl,ArticlesUrl_next
0,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...,Setting Up Your Profile,settingup,administration
1,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...,Basic Account Administration,administration,organazing
2,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...,Organizing Work With Spaces and Folders,organazing,projects
3,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...,Launching New Projects,projects,more
6,Getting Started,Step 3: Team Onboarding and Collaboration,https://help.wrike.com//hc/en-us/articles/2103...,Team Onboarding,onboarding,change
7,Getting Started,Step 3: Team Onboarding and Collaboration,https://help.wrike.com//hc/en-us/articles/2103...,Change Management,change,habits


In [102]:
df_mine.shape

(1431, 8)

In [89]:
from bs4 import BeautifulSoup, NavigableString, Tag
import numpy as np
def split_on_subtasks(row): 
    s1 = row["url"]
    resp = req.get(s1)
    s_pattern = BeautifulSoup(resp.text) 
    s = s_pattern.findAll(text=True)
    s_save = []
  
    if row["ArticlesUrl_next"] == '' or row["ArticlesUrl_next"] is None:
        new_tags2 = None 
    else:
        #print(row["between_two_2"])
        try: 
            new_tags2 = s_pattern.find(id=row["ArticlesUrl_next"]).parent
        except: 
            return []
    
    try: 
        new_tags = s_pattern.find(id=row["ArticlesUrl"]).parent
    except:
        return []
    
    save_ = False
    
    for i in s:
        #print("i", i)
        s1 = list(i.parents)
        #print(s1)
        if sum([i == new_tags2 for i in s1]) == 1:
            break
        #print(sum([i == new_tags2 for i in s1]))
        
        if save_:
            i = i.replace(g_text4, " ")
            i = re.sub('[\n\r/.]', '', i)
            i = re.sub(' +', ' ',i)
            s_save.append(i)
        
        #print("sdf", sum([i == new_tags2 for i in s1]))

        if sum([i == new_tags for i in s1]) == 1: 
            save_ = True
    #print(s_save)
    return s_save

In [90]:
df_mine.sample(5)

,First_stage,Second_stage,url,ArticlesName,ArticlesUrl,ArticlesUrl_next
536,"Tasks, Folders, Projects and Spaces",External Request Forms,https://help.wrike.com//hc/en-us/articles/1150...,Overview,overview,important
705,Work Views,Zoom In or Out on the Gantt Chart,https://help.wrike.com//hc/en-us/articles/2103...,Overview,overview,adjust
187,"Tasks, Folders, Projects and Spaces",Proofing,https://help.wrike.com//hc/en-us/articles/2103...,Wrike Proof Comment Export,export,video
320,"Tasks, Folders, Projects and Spaces",Custom Field Management,https://help.wrike.com//hc/en-us/articles/3600...,Remove a Custom Field,remove,article
68,"Tasks, Folders, Projects and Spaces",Subtasks,https://help.wrike.com//hc/en-us/articles/2103...,Important Information,important,create


In [91]:
%%time
df_mine["text"] = df_mine.apply(split_on_subtasks, axis = 1)

CPU times: user 1min 8s, sys: 1.16 s, total: 1min 9s
Wall time: 9min 25s


In [97]:
df_mine["text_len"] = df_mine["text"].apply(len)

In [98]:
df_mine.head()

,First_stage,Second_stage,url,ArticlesName,ArticlesUrl,ArticlesUrl_next,text,text_len
0,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...,Setting Up Your Profile,settingup,administration,"[, A personalized and up-to-date profile adds ...",18
1,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...,Basic Account Administration,administration,organazing,"[, Update the account setting that will affect...",13
2,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...,Organizing Work With Spaces and Folders,organazing,projects,"[, , Set up Spaces, , Spaces are hubs that sto...",44
3,Getting Started,Step 2: Setting Up Your Workspace,https://help.wrike.com//hc/en-us/articles/2103...,Launching New Projects,projects,more,"[, Whether you’re launching a new product, wri...",52
6,Getting Started,Step 3: Team Onboarding and Collaboration,https://help.wrike.com//hc/en-us/articles/2103...,Team Onboarding,onboarding,change,"[, You’ve spent some time choosing the best so...",150


In [100]:
df_mine["text_len"].max(), df_mine["text_len"].min()

(1098, 0)

In [103]:
df_mine_copy = df_mine[(df_mine.text_len != 0)]
df_mine_copy.shape, df_mine.shape

((1094, 8), (1431, 8))

In [105]:
df_mine.to_csv("help_title_mine.csv", index = False)

from sample 

In [ ]:
import pandas as pd
import re
df_help = pd.read_excel("help_title.xlsx")
df_help = df_help[["Раздел", "url_1", "Подраздел", "url_2", "Статья", "url_3", "Абзац", "url_4"]]
df_help.head()

In [ ]:
d_all_data = {}
d1 = df_help.set_index("url_4")["Абзац"].to_dict()
d2 = df_help.set_index("url_3")["Статья"].to_dict()
d3 = df_help.set_index("url_2")["Подраздел"].to_dict()
d4 = df_help.set_index("url_1")["Раздел"].to_dict()
d_all_data = {**d1,**d2,**d3,**d4}

In [ ]:
df_task = df_task.fillna('')
def find_https(x):
    s1 = ''
    for i in ["post_title", "post_text", "official_comments", "not_official_comments"]:
        if x[i] == '': 
            continue
        for j in x[i].split():
            if "http" in j and "user_images" not in j and "bit.ly" not in j: 
                j = re.sub('href=', '', j)
                j = re.sub('noreferrer">', '', j)
                j = re.sub('link:', '', j)
                j = re.sub('"' , '', j)
                j = re.sub('</a>' , '', j)
                s1 += j
                s1 += ','
    if s1 == '':
        return s1
    return s1.split(",")

In [ ]:
df_task[f"url_data"] = df_task[["post_title", "post_text", "official_comments", "not_official_comments"]].apply(find_https, axis = 1)

In [ ]:
df_task = df_task[df_task[f"url_data"] != ''].reset_index(drop=True)

In [ ]:
df_task = df_task.assign(url_task=[[(k, d_all_data[k]) for k in row if d_all_data.get(k)] for row in df_task.url_data])

In [ ]:
df_task["url_task"] = df_task["url_task"].apply(lambda x: None if x == [] else x[-1])
df_task = df_task.dropna(subset=["url_task"])
df_task["url_"] = df_task["url_task"].apply(lambda x: x[0])
df_task["names_"] = df_task["url_task"].apply(lambda x: x[-1])

In [ ]:
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
import math 
from nltk.corpus import wordnet

import pymorphy2
nltk.download('stopwords')
english_stopwords = stopwords.words("english")
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

def clean_data(x): 
    g_text1 = '\xa0'
    x = x.replace(g_text1, " ")
    x = re.sub('[\n\r]', '', x)
    x = re.sub(' +', ' ',x)
    return x 

In [ ]:
def preprocess_text(text, tokenizer=RegexpTokenizer(r'\w+'), 
                    lemmatizer=WordNetLemmatizer(), stopwords=stopwords.words("english")):
    tokens = tokenizer.tokenize(text)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if token not in english_stopwords\
              and token != " " \
              and not token.isdigit()]
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    clean_text = " ".join(lemmas)
    return clean_text

In [ ]:
df_task[["post_title", "post_text", "official_comments", "not_official_comments"]] = df_task[["post_title", "post_text", "official_comments", "not_official_comments"]].applymap(preprocess_text)

In [ ]:
df_task[["post_title", "post_text", "official_comments", "not_official_comments"]] = df_task[["post_title", "post_text", "official_comments", "not_official_comments"]].applymap(clean_data)